1-Chargement de l'ensemble de données de calibration de caméra+ parametres intra

In [3]:
import xml.etree.ElementTree as ET

def load_calibration_data(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract data
    camera_matrix = [float(val) for val in root.find('camera_matrix').text.split()]
    distortion_coefficients = [float(val) for val in root.find('distortion_coefficients').text.split()]
    object_points = [float(val) for val in root.find('object_points').text.split()]
    image_points = [float(val) for val in root.find('image_points').text.split()]

    return camera_matrix, distortion_coefficients, object_points, image_points

# Charger les données de calibration
camera_matrix, distortion_coefficients, object_points, image_points = load_calibration_data('chessboardd.xml')

# Extraire les paramètres de calibration de la caméra
print("Camera Matrix: ", camera_matrix)




Camera Matrix:  [0.28264900057285436, 0.5644754474187389, 0.07562297193269518, 0.6516823768873415, 0.17124689755725175, 0.5131796988181225, 0.05266466216904564, 0.3630292237277085, 0.20075858139271863]
Distortion Coefficients:  [0.4661006367512789, -0.49236304622387705, -0.27099991028977344, 0.13884784247024384, 0.4704005198998509]




# 2. paramètres extrinsèques (vecteurs de translation et de rotation).

mon script charge les données d'un fichier XML contenant des informations sur la calibration de la caméra, puis calcule les paramètres extrinsèques de la caméra (matrice de rotation et vecteur de translation) à partir des points d'objet et d'image correspondants.

## Librairies Utilisées

- `cv2` (OpenCV) : pour les fonctions de traitement d'image et de calcul de la matrice d'homographie.
- `numpy` : pour la manipulation de tableaux et les calculs numériques.
- `xml.etree.ElementTree` : pour l'analyse XML.

## Entrées

- Le chemin complet vers le fichier XML contenant les données de calibration de la caméra.

## Sorties

- Vecteur de translation : Le vecteur de translation de la caméra par rapport à la scène.
- Matrice de rotation : La matrice de rotation de la caméra par rapport à la scène.

## Fonctions Principales

- `parse` : Charge les données du fichier XML.
- `findHomography` : Calcul de la matrice d'homographie.
- `np.linalg.inv` : Inverse de la matrice de la caméra.
- `np.linalg.svd` : Décomposition en valeurs singulières pour normaliser la matrice de rotation.
- `np.dot` : Produit scalaire de vecteurs/matrices.


## Remarques

- Le fichier XML  contenir les données de calibration de la caméra, y compris la matrice de la caméra, les coefficients de distorsion, les points d'objet et les points d'image .
- Le script suppose que le nombre de points d'objet et d'image est le même et qu'il y a au moins 4 points pour calculer la matrice d'homographie.





In [4]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Spécifier le chemin complet vers le fichier XML
xml_file_path = r'chessboardd.xml'

# Charger les données du fichier XML
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Extraire les données
camera_matrix_data = np.array([float(x) for x in root.find('camera_matrix').text.split()]).reshape(3, 3)
distortion_coefficients_data = np.array([float(x) for x in root.find('distortion_coefficients').text.split()])
object_points_data = np.array([[float(y) for y in x.text.split()] for x in root.find('object_points').findall('point')])
image_points_data = np.array([[float(y) for y in x.text.split()] for x in root.find('image_points').findall('point')])

# Vérifier les dimensions des vecteurs de points
assert object_points_data.shape[0] == image_points_data.shape[0], "Le nombre de points d'objet et d'image doit être le même."

# Vérifier le nombre de points
num_points = object_points_data.shape[0]
assert num_points >= 4, "Il doit y avoir au moins 4 points correspondants pour calculer la matrice d'homographie."


H, _ = cv2.findHomography(object_points_data[:,:2], image_points_data)

camera_matrix_inv = np.linalg.inv(camera_matrix_data)
H_norm = np.dot(camera_matrix_inv, H)
U, S, Vt = np.linalg.svd(H_norm)
lambda_ = 1 / np.linalg.norm(U[:,0])
U *= lambda_
Vt *= lambda_
W = np.array([[0,-1,0],[1,0,0],[0,0,1]], dtype=float)
R = np.dot(np.dot(U, W.T), Vt)
t = np.dot(camera_matrix_inv, H_norm[:,2])

# Afficher les paramètres externes
print("Vecteurs de translation :\n", t)
print("\nMatrice de rotation :\n", R)

Vecteurs de translation :
 [-13.06690306   1.0769582   18.41643724]

Matrice de rotation :
 [[-0.65727377 -0.51360712  0.55154231]
 [ 0.0709789   0.68638783  0.7237636 ]
 [-0.75030206  0.5148587  -0.41468945]]


# 3.a Documentation de l'Algorithme genetique d'Optimisation des Paramètres de Caméra

on utilise un algorithme génétique pour optimiser les paramètres intrinsèques de la caméra (matrice de caméra et coefficients de distorsion) en minimisant l'erreur de projection entre les points d'objet et d'image correspondants.

## Fonctions Principales

### `load_camera_data(xml_file_path)`
Charge les données du fichier XML contenant les informations de calibration de la caméra.

- **Entrée :** Le chemin complet vers le fichier XML.
- **Sortie :** Un tuple contenant les données de la matrice de la caméra, les coefficients de distorsion, les points d'objet et les points d'image correspondants.

### `fitness(parameters, object_points_data, image_points_data)`
Calcule la fonction de fitness en évaluant l'erreur de projection entre les points d'objet et d'image.

- **Entrées :**
    - `parameters` : Les paramètres de la caméra à évaluer.
    - `object_points_data` : Les points d'objet 3D.
    - `image_points_data` : Les points d'image 2D correspondants.
- **Sortie :** L'erreur quadratique moyenne (MSE) entre les points d'image projetés et les points d'image réels.

### `genetic_algorithm(population_size, num_generations, object_points_data, image_points_data)`
Applique l'algorithme génétique pour optimiser les paramètres de la caméra.

- **Entrées :**
    - `population_size` : La taille de la population d'individus.
    - `num_generations` : Le nombre de générations à exécuter.
    - `object_points_data` : Les points d'objet 3D.
    - `image_points_data` : Les points d'image 2D correspondants.
- **Sortie :** Les meilleurs paramètres trouvés par l'algorithme.



## Remarques

- Cet algorithme utilise une approche d'optimisation basée sur une population d'individus.
- La fonction de fitness est définie comme l'erreur quadratique moyenne entre les points d'image projetés et les points d'image réels.
- L'algorithme génétique effectue une sélection des meilleurs individus, un croisement et une mutation pour générer une nouvelle population à chaque génération.
- Les paramètres de la caméra sont représentés sous forme d'un vecteur combinant la matrice de la caméra et les coefficients de distorsion.


In [5]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Charger les données du fichier XML
def load_camera_data(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    camera_matrix_data = np.array([float(x) for x in root.find('camera_matrix').text.split()]).reshape(3, 3)
    distortion_coefficients_data = np.array([float(x) for x in root.find('distortion_coefficients').text.split()])
    object_points_data = np.array([[float(y) for y in x.text.split()] for x in root.find('object_points').findall('point')])
    image_points_data = np.array([[float(y) for y in x.text.split()] for x in root.find('image_points').findall('point')])

    return camera_matrix_data, distortion_coefficients_data, object_points_data, image_points_data
# Fitness function
def fitness(parameters, object_points_data, image_points_data):
    # Extraire les paramètres
    camera_matrix = parameters[:9].reshape(3, 3)
    distortion_coefficients = parameters[9:]

    # Projet des points d'objet sur le plan image
    _, rvecs, tvecs = cv2.solvePnP(object_points_data, image_points_data, camera_matrix, distortion_coefficients)

    projected_points, _ = cv2.projectPoints(object_points_data, rvecs, tvecs, camera_matrix, distortion_coefficients)

    # Calculer l'erreur quadratique moyenne
    mse = np.mean(np.square(image_points_data - projected_points.squeeze()))
    
    return mse

# Algorithme génétique
def genetic_algorithm(population_size, num_generations, object_points_data, image_points_data):
    population = []

    for _ in range(population_size):
        # Générer des paramètres de caméra et de distorsion aléatoires
        camera_matrix = np.random.rand(3, 3)  # Matrice de caméra (3x3)
        distortion_coefficients = np.random.rand(5)  # Coefficients de distorsion (5,)
        population.append(np.concatenate((camera_matrix.flatten(), distortion_coefficients)))

    for generation in range(num_generations):
        # Calculer la fitness pour chaque individu dans la population
        fitness_values = [fitness(parameters, object_points_data, image_points_data) for parameters in population]

        # Sélectionner les meilleurs individus
        num_parents = max(int(population_size * 0.2), 2)  # Assurez-vous qu'il y a au moins 2 parents
        parent_indices = np.argsort(fitness_values)[:num_parents]  # Obtenir les indices des meilleurs individus
        parents = [population[i] for i in parent_indices]

        # Croisement
        offsprings = []
        for _ in range(population_size - num_parents):
            # Sélectionner deux indices uniques à partir des indices des parents
            index1, index2 = np.random.choice(num_parents, 2, replace=False)
            parent1, parent2 = parents[index1], parents[index2]
            crossover_point = np.random.randint(14)
            offspring = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            offsprings.append(offspring)

        # Mutation
        mutation_rate = 0.1
        for i in range(num_parents, population_size):
            if np.random.rand() < mutation_rate:
                mutated_gene_index = np.random.randint(14)
                offsprings[i - num_parents][mutated_gene_index] += np.random.normal(0, 0.1)

        # Mettre à jour la population
        population = parents + offsprings

        # Afficher les détails de la génération actuelle
        best_fitness = min(fitness_values)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

    # Retourner la meilleure solution trouvée
    best_solution_index = np.argmin(fitness_values)
    return population[best_solution_index]

# Charger les données du fichier XML
xml_file_path = r'chessboardd.xml'
_, _, object_points_data, image_points_data = load_camera_data(xml_file_path)

# Algorithme génétique parameters
population_size = 50
num_generations = 1

# Exécuter l'algorithme génétique
best_parameters = genetic_algorithm(population_size, num_generations, object_points_data, image_points_data)

# Afficher les paramètres optimisés
print("Paramètres Optimisés de la Caméra:")
print("Matrice de caméra :\n", best_parameters[:9].reshape(3, 3))
print("\nCoefficients de distorsion :\n", best_parameters[9:])

Generation 1: Best Fitness = 0.017341315134829574
Paramètres Optimisés de la Caméra:
Matrice de caméra :
 [[0.56562718 0.45087725 0.3967345 ]
 [0.30503643 0.65192495 0.02705023]
 [0.63985188 0.21126748 0.20700707]]

Coefficients de distorsion :
 [0.44085576 0.55667516 0.69199488 0.27909579 0.89905584]


# 3.b Documentation de l'Algorithme d'Optimisation des Paramètres Extrinsèques de Caméra

mon script utilise un algorithme génétique pour optimiser les paramètres extrinsèques de la caméra (vecteurs de translation et matrice de rotation) en minimisant l'erreur de projection entre les points d'objet et d'image correspondants.

## Changement de Paramètres

Dans cette version, le script a été modifié pour inclure les paramètres extrinsèques de la caméra (vecteurs de translation et matrice de rotation) dans le processus d'optimisation. Les fonctions `fitness` et `genetic_algorithm` ont été mises à jour pour prendre en compte ces nouveaux paramètres.

Pour plus de détails sur les fonctions principales, consultez la documentation précédente.


In [7]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Charger les données du fichier XML
def load_camera_data(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    camera_matrix_data = np.array([float(x) for x in root.find('camera_matrix').text.split()]).reshape(3, 3)
    distortion_coefficients_data = np.array([float(x) for x in root.find('distortion_coefficients').text.split()])
    object_points_data = np.array([[float(y) for y in x.text.split()] for x in root.find('object_points').findall('point')])
    image_points_data = np.array([[float(y) for y in x.text.split()] for x in root.find('image_points').findall('point')])

    return camera_matrix_data, distortion_coefficients_data, object_points_data, image_points_data

# Fitness function
def fitness(parameters, object_points_data, image_points_data, camera_matrix_data, distortion_coefficients_data):
    # Extraire les paramètres
    t, R = parameters

    # Projet des points d'objet sur le plan image
    projected_points, _ = cv2.projectPoints(object_points_data, R, t, camera_matrix_data, distortion_coefficients_data)

    # Calculer l'erreur quadratique moyenne
    mse = np.mean(np.square(image_points_data - projected_points.squeeze()))
    
    return mse

# Algorithme génétique
def genetic_algorithm(population_size, num_generations, object_points_data, image_points_data, camera_matrix_data, distortion_coefficients_data):
    population = []

    for _ in range(population_size):
        # Générer des paramètres de translation et rotation aléatoires
        t = np.random.rand(3, 1)  # Vecteur de translation (3x1)
        R = np.random.rand(3, 3)   # Matrice de rotation (3x3)
        population.append((t, R))

    for generation in range(num_generations):
        # Calculer la fitness pour chaque individu dans la population
        fitness_values = [fitness(parameters, object_points_data, image_points_data, camera_matrix_data, distortion_coefficients_data) for parameters in population]

        # Sélectionner les meilleurs individus
        num_parents = max(int(population_size * 0.2), 2)  # Assurez-vous qu'il y a au moins 2 parents
        parent_indices = np.argsort(fitness_values)[:num_parents]  # Obtenir les indices des meilleurs individus
        parents = [population[i] for i in parent_indices]

        # Croisement
        offsprings = []
        for _ in range(population_size - num_parents):
            # Sélectionner deux indices uniques à partir des indices des parents
            index1, index2 = np.random.choice(num_parents, 2, replace=False)
            parent1, parent2 = parents[index1], parents[index2]
            crossover_point = np.random.randint(3)
            offspring = (np.concatenate((parent1[0][:crossover_point], parent2[0][crossover_point:]), axis=0), parent1[1])
            offsprings.append(offspring)

        # Mutation
        mutation_rate = 0.1
        for i in range(num_parents, population_size):
            if np.random.rand() < mutation_rate:
                mutated_gene_index = np.random.randint(3)
                offsprings[i - num_parents][0][mutated_gene_index] += np.random.normal(0, 0.1)

        # Mettre à jour la population
        population = parents + offsprings

        # Afficher les détails de la génération actuelle
        best_fitness = min(fitness_values)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

    # Retourner la meilleure solution trouvée
    best_solution_index = np.argmin(fitness_values)
    return population[best_solution_index]

# Charger les données du fichier XML
xml_file_path = r'chessboardd.xml'
camera_matrix_data, distortion_coefficients_data, object_points_data, image_points_data = load_camera_data(xml_file_path)

# Algorithme génétique parameters
population_size = 50
num_generations = 1

# Exécuter l'algorithme génétique
best_parameters = genetic_algorithm(population_size, num_generations, object_points_data, image_points_data, camera_matrix_data, distortion_coefficients_data)

# Afficher les paramètres optimisés
print("Paramètres Optimisés de la Caméra:")
print("Vecteurs de translation :\n", best_parameters[0])
print("\nMatrice de rotation :\n", best_parameters[1])



Generation 1: Best Fitness = 0.05885131167888163
Paramètres Optimisés de la Caméra:
Vecteurs de translation :
 [[0.8109912 ]
 [0.17339458]
 [0.97012658]]

Matrice de rotation :
 [[0.7740631  0.00264582 0.68458587]
 [0.48321897 0.49683837 0.25756081]
 [0.48414756 0.1932206  0.8585353 ]]
